In [1]:
import chess
import chess.engine
import random
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6600506300522068142
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6254755840
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17637554184408226269
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
# comprobamos que la GPU está disponible

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = board3d
  for _ in range(conv_depth):
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_first')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

Skip connections (residual network) will likely improve the model for deeper connections.

In [12]:
def build_model_residual(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', data_format='channels_first')(board3d)
  for _ in range(conv_depth):
    previous = x
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', data_format='channels_first')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', data_format='channels_first')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, previous])
    x = layers.Activation('relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

In [13]:
model = build_model_residual(64, 5)
utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=False)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# Training ⬇️

In [6]:
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	container = np.load(r'C:\Users\Usuario\Desktop\Data Science Projects\Chess_Engine_TFM\data\random_generated\random_boards_d10.npz', allow_pickle=True)
	b, v, f = container['board_matrix'], container['eval'], container['board_fen']
	v[v == None] = 0
	v = np.asarray(v / abs(v).max() / 2 + 0.5, dtype=np.float32) # normalization (0 - 1)
	return b, v


x_train, y_train = get_dataset()
print(x_train.shape)
print(y_train.shape)

(1500000, 14, 8, 8)
(1500000,)


In [15]:
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 14, 8, 8)]   0           []                               
                                                                                                  
 conv2d_9 (Conv2D)              (None, 64, 8, 8)     8128        ['input_3[0][0]']                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 64, 8, 8)     36928       ['conv2d_9[0][0]']               
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 8, 8)    32          ['conv2d_10[0][0]']              
 alization)                                                                                 

In [10]:
model.fit(x_train, y_train,
          batch_size=2048,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])

model.save('./engine_models/depth10_model.h5')

Epoch 1/1000
660/660 [==============================] - 22s 23ms/step - loss: 0.0026 - val_loss: 0.0021 - lr: 5.0000e-04
Epoch 2/1000
660/660 [==============================] - 15s 22ms/step - loss: 0.0021 - val_loss: 0.0021 - lr: 5.0000e-04
Epoch 3/1000
660/660 [==============================] - 15s 23ms/step - loss: 0.0020 - val_loss: 0.0020 - lr: 5.0000e-04
Epoch 4/1000
660/660 [==============================] - 17s 26ms/step - loss: 0.0020 - val_loss: 0.0019 - lr: 5.0000e-04
Epoch 5/1000
660/660 [==============================] - 18s 27ms/step - loss: 0.0019 - val_loss: 0.0021 - lr: 5.0000e-04
Epoch 6/1000
660/660 [==============================] - 18s 27ms/step - loss: 0.0019 - val_loss: 0.0020 - lr: 5.0000e-04
Epoch 7/1000
660/660 [==============================] - 18s 27ms/step - loss: 0.0019 - val_loss: 0.0019 - lr: 5.0000e-04
Epoch 8/1000
660/660 [==============================] - 18s 27ms/step - loss: 0.0019 - val_loss: 0.0020 - lr: 5.0000e-04
Epoch 9/1000
660/660 [==========

In [16]:
model.fit(x_train, y_train,
          batch_size=2048,
          epochs=1000,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])

model.save('./engine_models/depth10_model_residuals.h5')

Epoch 1/1000
660/660 [==============================] - 54s 79ms/step - loss: 0.0311 - val_loss: 0.0052 - lr: 5.0000e-04
Epoch 2/1000
660/660 [==============================] - 52s 79ms/step - loss: 0.0036 - val_loss: 0.0029 - lr: 5.0000e-04
Epoch 3/1000
660/660 [==============================] - 52s 79ms/step - loss: 0.0028 - val_loss: 0.0024 - lr: 5.0000e-04
Epoch 4/1000
660/660 [==============================] - 52s 79ms/step - loss: 0.0025 - val_loss: 0.0024 - lr: 5.0000e-04
Epoch 5/1000
660/660 [==============================] - 53s 80ms/step - loss: 0.0024 - val_loss: 0.0027 - lr: 5.0000e-04
Epoch 6/1000
660/660 [==============================] - 52s 79ms/step - loss: 0.0024 - val_loss: 0.0023 - lr: 5.0000e-04
Epoch 7/1000
660/660 [==============================] - 53s 80ms/step - loss: 0.0023 - val_loss: 0.0023 - lr: 5.0000e-04
Epoch 8/1000
660/660 [==============================] - 53s 80ms/step - loss: 0.0023 - val_loss: 0.0023 - lr: 5.0000e-04
Epoch 9/1000
660/660 [==========